In [1]:
from sklearn import datasets
import numpy as np
import torch

In [3]:
def calc_similarity(data, gamma, neighbors):
    distances = torch.cdist(data, data)**2
    similarity = torch.exp(-gamma * distances)
    similarity[range(len(data)), range(len(data))] = 0
    _, index = similarity.topk(neighbors, dim=1)
    mask = torch.zeros_like(similarity)
    mask = torch.scatter(mask, 1, index, 1)
    similarity = similarity * mask
    return similarity

In [4]:
def calc_symmetric_laplacian(similarity):
    degree = similarity.sum(dim=1)
    laplacian = degree.diag() - similarity
    inv_degree = degree ** (-0.5)
    symmetric_laplacian = (inv_degree.diag()).mm(laplacian).mm(inv_degree.diag())
    return symmetric_laplacian

In [5]:
def get_spectral_embedding(symmetric_laplacian, K):
    eigvalues, eigvectors = torch.linalg.eig(symmetric_laplacian)
    _, index = eigvalues.real.topk(K, largest=False)
    embedding = eigvectors.real[:, index]
    embedding = embedding / (embedding.norm(dim=1, keepdim=True) + 1e-10)
    return embedding

In [6]:
N = 1500
K = 3
gamma = 0.5
neighbors = 10

In [7]:
X, y = datasets.make_blobs(n_samples=N, random_state=170)
transformation = [[0.6, 0.6], [-0.4, -0.8]]
data = np.dot(X, transformation)

In [8]:
data = torch.from_numpy(data).float()
similarity = calc_similarity(data, gamma, neighbors)
symmetric_laplacian = calc_symmetric_laplacian(similarity)
embedding = get_spectral_embedding(symmetric_laplacian, K)

In [9]:
def kmeans(data, N, K):
    centers = data[torch.randint(0, N, (K,))]
    iters = 200
    for i in range(iters):
        dist = torch.cdist(data, centers) ** 2
        assign_idx = dist.argmin(dim=1, keepdim=True)
        centers.zero_()
        centers = torch.scatter_add(centers, 0, assign_idx.repeat(1, data.shape[1]), data)
        counts = torch.zeros(K, 1)
        counts = torch.scatter_add(counts, 0, assign_idx, torch.ones(N, 1))
        centers /= counts + 1e-10
    return centers, assign_idx

In [10]:
centers, assign_idx = kmeans(embedding, N, K)
rcenters, rassign_idx = kmeans(data, N, K)